In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os

# Ruta del archivo CSV
base_path = r'C:\Users\JP\Desktop\Archivos para Analisis de Sentimientos\Nuevo Analisis de Sentimientos\Producto con sentimiento ind y prom con ID'
csv_path = os.path.join(base_path, 'productos_con_id.csv')

# Cargar el dataset
df_productos = pd.read_csv(csv_path)

# Crear una nueva columna combinando características relevantes
df_productos['caracteristicas'] = (
    df_productos['Producto'] + " " + 
    df_productos['Categoría'] + " " + 
    df_productos['Sentimiento promedio'].astype(str)
)

# Vectorización de texto usando TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
matriz_tfidf = vectorizer.fit_transform(df_productos['caracteristicas'])

# Calcular la similitud de coseno entre los productos
similitud_coseno = cosine_similarity(matriz_tfidf, matriz_tfidf)

# Función para obtener recomendaciones
def recomendar_producto(producto_id, top_n=5):
    try:
        # Índice del producto dado
        idx = df_productos[df_productos['ID'] == producto_id].index[0]
        
        # Similaridades del producto con los demás
        similitudes = list(enumerate(similitud_coseno[idx]))
        
        # Ordenar por similitud (de mayor a menor)
        similitudes = sorted(similitudes, key=lambda x: x[1], reverse=True)
        
        # Obtener los top_n productos más similares (excluyendo el mismo producto)
        productos_similares = [i[0] for i in similitudes[1:top_n+1]]
        
        return df_productos.iloc[productos_similares][['ID', 'Producto', 'Categoría', 'Sentimiento promedio']]
    except IndexError:
        return pd.DataFrame(columns=['ID', 'Producto', 'Categoría', 'Sentimiento promedio'])

# Probar con un producto específico
producto_id = 1  # Cambiar al ID de un producto real
recomendaciones = recomendar_producto(producto_id, top_n=5)

# Mostrar recomendaciones
print(f"Recomendaciones para el producto con ID {producto_id}:")
print(recomendaciones)

# Generar y guardar recomendaciones para todos los productos
todas_recomendaciones = []

for producto_id in df_productos['ID']:
    recomendaciones = recomendar_producto(producto_id, top_n=5)
    for _, rec in recomendaciones.iterrows():
        todas_recomendaciones.append({
            'Producto Base ID': producto_id,
            'Recomendado ID': rec['ID'],
            'Recomendado Producto': rec['Producto'],
            'Categoría': rec['Categoría'],
            'Sentimiento promedio': rec['Sentimiento promedio']
        })

# Crear un DataFrame para las recomendaciones
df_recomendaciones = pd.DataFrame(todas_recomendaciones)

# Guardar recomendaciones en CSV
recomendaciones_csv_path = os.path.join(base_path, 'recomendaciones.csv')
df_recomendaciones.to_csv(recomendaciones_csv_path, index=False)

# Mensaje de confirmación
print(f"Recomendaciones guardadas en: {recomendaciones_csv_path}")


Recomendaciones para el producto con ID 1:
    ID                                           Producto Categoría  \
60  61  HP Pavilion x360 14 inch Laptop, FHD Touchscre...   laptops   
54  55  HP 14 Laptop, Intel Celeron N4020, 4 GB RAM, 6...   laptops   
31  32  HP Chromebook 14 inch Laptop, FHD Display, Int...   laptops   
5    6  HP Stream 14" HD BrightView Laptop, Intel Cele...   laptops   
2    3  ASUS 2024 Vivobook Go 11.6” HD Laptop, Intel D...   laptops   

    Sentimiento promedio  
60              0.409127  
54             -0.065625  
31              0.443981  
5               0.314484  
2               0.107833  
Recomendaciones guardadas en: C:\Users\JP\Desktop\Archivos para Analisis de Sentimientos\Nuevo Analisis de Sentimientos\Producto con sentimiento ind y prom con ID\recomendaciones.csv
